In [26]:
import pandas as pd
from pathlib import Path
import zipfile
import shutil
import os

In [136]:
SRC = Path(__file__).parent.resolve()#Jupyter去掉_file_
BLD = SRC.joinpath("bld").resolve()


In [32]:
csv_file_paths = []
for root, dirs, files in os.walk(folder_path):  # 遍历所有子目录
    for file in files:
         if file.endswith('.csv'):  # 只处理 CSV 文件
            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")  # 打印当前处理的文件路径
            csv_file_paths.append(file_path)


Processing file: /Users/hengdegao/python/thesis/data/csmar/股东治理/关联交易情况文件/RPT_Operation1.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/股东治理/关联交易情况文件/RPT_Operation.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/股东治理/股东大会/CG_Shrmeet.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/股东治理/高管兼任-用董事会治理的表/CG_Rzgddw1.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/股东治理/高管兼任-用董事会治理的表/CG_Rzgddw.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/管理层治理/股权激励方案表024904107(仅供vip使用)/CG_EIPLAN.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/管理层治理/高管人数、持股及薪酬情况表/CG_ManagerShareSalary.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/基本信息/STK_LISTEDCOINFOANL.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/基本信息/利润表000222262(仅供vip使用)/FS_Comins.csv
Processing file: /Users/hengdegao/python/thesis/data/csmar/基本信息/subsidiary_company/FN_Fn061.csv
Processing file: /Users/hengdegao/python/thesis/data/c

In [33]:
csv_file_paths

['/Users/hengdegao/python/thesis/data/csmar/股东治理/关联交易情况文件/RPT_Operation1.csv',
 '/Users/hengdegao/python/thesis/data/csmar/股东治理/关联交易情况文件/RPT_Operation.csv',
 '/Users/hengdegao/python/thesis/data/csmar/股东治理/股东大会/CG_Shrmeet.csv',
 '/Users/hengdegao/python/thesis/data/csmar/股东治理/高管兼任-用董事会治理的表/CG_Rzgddw1.csv',
 '/Users/hengdegao/python/thesis/data/csmar/股东治理/高管兼任-用董事会治理的表/CG_Rzgddw.csv',
 '/Users/hengdegao/python/thesis/data/csmar/管理层治理/股权激励方案表024904107(仅供vip使用)/CG_EIPLAN.csv',
 '/Users/hengdegao/python/thesis/data/csmar/管理层治理/高管人数、持股及薪酬情况表/CG_ManagerShareSalary.csv',
 '/Users/hengdegao/python/thesis/data/csmar/基本信息/STK_LISTEDCOINFOANL.csv',
 '/Users/hengdegao/python/thesis/data/csmar/基本信息/利润表000222262(仅供vip使用)/FS_Comins.csv',
 '/Users/hengdegao/python/thesis/data/csmar/基本信息/subsidiary_company/FN_Fn061.csv',
 '/Users/hengdegao/python/thesis/data/csmar/信息披露/上市公司信息披露考评信息表/AIQ_LCInfoDisEvalY.csv',
 '/Users/hengdegao/python/thesis/data/csmar/信息披露/业绩预告与定期报告对比表(季)231800773(仅供vip使用)/BF_FORECAST.c

In [27]:
folder_path = os.getcwd()  # 替换为你的文件夹路径
folder_path

'/Users/hengdegao/python/thesis/data'

In [137]:
import pandas as pd

# 定义列和筛选条件
columns = ['Stkcd', 'Reptdt', 'Curtype', 'Isam', 'Repat','Content','Kind']
priority_repat = [1, 3, 8, 9, 10]

# 文件 1：处理 Isam 无问题的文件
df_chunks1 = pd.read_csv(
    SRC/'original_data/csmar/股东治理/关联交易情况文件/RPT_Operation1.csv',
    usecols=columns,
    dtype={'Stkcd': str, 'Isam': float},  # Isam 直接读取为 float
    chunksize=1000
)

# 文件 2：处理可能有 Isam 类型问题的文件
df_chunks2 = pd.read_csv(
    SRC/'original_data/csmar/股东治理/关联交易情况文件/RPT_Operation.csv',
    usecols=columns,
    dtype={'Stkcd': str},  # Isam 先读取为默认类型
    chunksize=1000
)

# 合并所有块
combined_chunks = []

# 处理文件 1
for chunk in df_chunks1:
    # 筛选符合条件的数据
    filtered_chunk = chunk[chunk["Repat"].isin(priority_repat)]
    combined_chunks.append(filtered_chunk)

# 处理文件 2
for chunk in df_chunks2:
    # 筛选符合条件的数据
    filtered_chunk = chunk[chunk["Repat"].isin(priority_repat)].copy()
    # 将 Isam 转换为数值类型，非法值处理为 NaN
    filtered_chunk.loc[:, 'Isam'] = pd.to_numeric(filtered_chunk['Isam'], errors='coerce')
    combined_chunks.append(filtered_chunk)

# 合并两个文件的结果
final_df = pd.concat(combined_chunks, ignore_index=True)
final_df['Isam'] = final_df['Isam'].astype(float)


# 查看结果
print(final_df.head())
print(final_df.dtypes)

# 如果需要保存为新的文件


    Stkcd      Reptdt  Repat    Kind Curtype          Isam  \
0  600760  2018-12-31    1.0    采购商品     CNY  9.750761e+09   
1  600760  2018-12-31    3.0    接受劳务     CNY  3.809900e+08   
2  600760  2018-12-31    3.0    提供劳务     CNY  4.845132e+06   
3  600760  2018-12-31    1.0    销售商品     CNY  1.464491e+09   
4  600760  2018-12-31    3.0  接受金融服务     NaN           NaN   

                      Content  
0                        采购商品  
1                        接受劳务  
2                        提供劳务  
3                        销售商品  
4  公司接受中航工业集团财务有限责任公司提供的金融服务。  
Stkcd       object
Reptdt      object
Repat      float64
Kind        object
Curtype     object
Isam       float64
Content     object
dtype: object


In [138]:
print(final_df['Isam'].dtypes)  # 应输出 float64
usd_to_cny_rate = 7.0

# 只转换 Curtype 为 USD 的数据
final_df.loc[final_df['Curtype'] == 'USD', 'Isam'] *= usd_to_cny_rate

# 更新 Curtype 为 CNY
final_df.loc[final_df['Curtype'] == 'USD', 'Curtype'] = 'CNY'

float64


In [139]:

filtered_df = final_df.loc[final_df["Curtype"] == "CNY"]

# 将 Reptdt 列只保留年份
filtered_df.loc[:, "Reptdt"] = pd.to_datetime(filtered_df["Reptdt"]).dt.year

# 输出筛选结果
filtered_df = filtered_df[["Stkcd", "Reptdt", "Isam"]]
print(filtered_df)


         Stkcd Reptdt          Isam
0       600760   2018  9.750761e+09
1       600760   2018  3.809900e+08
2       600760   2018  4.845132e+06
3       600760   2018  1.464491e+09
5       600760   2019  1.213288e+10
...        ...    ...           ...
484546  600760   2017  2.020000e+06
484547  600760   2017  6.911242e+09
484548  600760   2017  3.862585e+08
484549  600760   2017  3.872981e+06
484550  600760   2017  1.602987e+09

[479016 rows x 3 columns]


In [140]:
# 按公司和年份分组并求和
summed_df = filtered_df.groupby(['Stkcd', 'Reptdt'], as_index=False)['Isam'].sum()

# 输出结果
print(summed_df)


        Stkcd Reptdt          Isam
0      000002   2014  1.701230e+06
1      000002   2017  5.378190e+08
2      000002   2019  5.995200e+07
3      000002   2020  3.742149e+09
4      000002   2021  4.643965e+09
...       ...    ...           ...
28417  873593   2023  3.716975e+07
28418  873665   2023  2.529465e+06
28419  873679   2023  8.531000e+04
28420  873806   2023  5.104297e+08
28421  873833   2023  5.515182e+07

[28422 rows x 3 columns]


In [141]:
summed_df = summed_df[(summed_df['Stkcd'] >= "000001") & (summed_df['Stkcd'] <= "679999")]



In [142]:
summed_df.nunique()

Stkcd      3972
Reptdt       10
Isam      26693
dtype: int64

In [143]:
summed_df

,Stkcd,Reptdt,Isam
0,000002,2014,1.701230e+06
1,000002,2017,5.378190e+08
2,000002,2019,5.995200e+07
3,000002,2020,3.742149e+09
4,000002,2021,4.643965e+09
...,...,...,...
26831,605598,2022,6.351754e+05
26832,605598,2023,2.462859e+05
26833,605599,2021,1.654594e+06
26834,605599,2022,1.302172e+06


In [127]:
# 汇总各货币的金额
currency_totals = final_df.groupby('Curtype')['Isam'].sum()

# 按金额排序并打印
print("各货币类型的总金额（降序）：")
print(currency_totals.sort_values(ascending=False))

# 检查是否存在较小的总金额（例如低于10,000）
threshold = 10000  # 设置忽略的阈值
small_currencies = currency_totals[currency_totals < threshold]
print("\n以下货币的总金额低于阈值，可以忽略：")
print(small_currencies)


各货币类型的总金额（降序）：
Curtype
CNY    1.167981e+14
PHP    7.785448e+09
XOF    6.421000e+09
KZT    3.383812e+09
JPY    4.290055e+08
EUR    1.114650e+08
ZAR    6.977471e+07
HKD    6.250000e+07
MYR    4.345600e+07
NZD    3.572765e+06
TWD    6.300000e+05
AUD    1.303868e+05
13     0.000000e+00
Name: Isam, dtype: float64

以下货币的总金额低于阈值，可以忽略：
Curtype
13    0.0
Name: Isam, dtype: float64


In [117]:
# 筛选 USD 的大金额数据
threshold = 1e8  # 设置筛选阈值（如 1 亿）
usd_large = final_df[(final_df['Curtype'] == 'USD') & (final_df['Isam'] > threshold)]

# 查看筛选结果
print(f"筛选出 {len(usd_large)} 条 USD 大金额数据")
print(usd_large.head())

筛选出 23 条 USD 大金额数据
        Stkcd      Reptdt  Repat                  Kind Curtype         Isam  \
7015   600801  2018-12-31    3.0            提供工程施工项目服务     USD  245000000.0   
65799  601727  2018-12-31    3.0  提供2x660MW坑口燃煤发电站供货服务     USD  147000000.0   
65800  601727  2018-12-31    3.0  提供2x660MW坑口燃煤发电站建设服务     USD  408600000.0   
65801  601727  2018-12-31    3.0  提供2x660MW坑口燃煤发电站供货服务     USD  806400000.0   
65812  601727  2019-12-31    3.0                提供离岸服务     USD  467956549.0   

                                           Content  
7015        本公司向Holcim Philippines,Inc.提供工程施工项目服务。  
65799  上海电气香港有限公司向坦塔尔煤田电站项目公司提供2x660MW坑口燃煤发电站供货服务。  
65800  上海电气香港有限公司向坦塔尔煤田电站项目公司提供2x660MW坑口燃煤发电站建设服务。  
65801          公司向坦塔尔煤田电站项目公司提供2x660MW坑口燃煤发电站供货服务。  
65812           本公司与上海电气香港国际工程公司向华信资源责任有限公司提供离岸服务。  


In [118]:
usd_large

,Stkcd,Reptdt,Repat,Kind,Curtype,Isam,Content
7015,600801,2018-12-31,3.0,提供工程施工项目服务,USD,2.450000e+08,"本公司向Holcim Philippines,Inc.提供工程施工项目服务。"
65799,601727,2018-12-31,3.0,提供2x660MW坑口燃煤发电站供货服务,USD,1.470000e+08,上海电气香港有限公司向坦塔尔煤田电站项目公司提供2x660MW坑口燃煤发电站供货服务。
65800,601727,2018-12-31,3.0,提供2x660MW坑口燃煤发电站建设服务,USD,4.086000e+08,上海电气香港有限公司向坦塔尔煤田电站项目公司提供2x660MW坑口燃煤发电站建设服务。
65801,601727,2018-12-31,3.0,提供2x660MW坑口燃煤发电站供货服务,USD,8.064000e+08,公司向坦塔尔煤田电站项目公司提供2x660MW坑口燃煤发电站供货服务。
65812,601727,2019-12-31,3.0,提供离岸服务,USD,4.679565e+08,本公司与上海电气香港国际工程公司向华信资源责任有限公司提供离岸服务。
65813,601727,2019-12-31,3.0,提供在岸服务,USD,4.347504e+08,上海电气香港国际工程公司向华信资源责任有限公司提供在岸服务。
70884,601866,2020-12-31,1.0,购买新船,USD,1.347200e+08,Oriental Fleet Pulp 01 Limited向大连中远海运重工有限公司购买新...
76108,601919,2015-12-31,1.0,采购集装箱,USD,1.688330e+08,采购集装箱
76169,601919,2020-12-31,1.0,购买23000TEU型集装箱船舶,USD,4.670400e+08,东方海外(国际)有限公司的全资附属单船公司向南通中远海运川崎船舶工程有限公司购买23000T...
76170,601919,2020-12-31,1.0,采购23000TEU型集装箱船舶,USD,3.113600e+08,东方海外(国际)有限公司的全资附属单船公司向大连中远海运川崎船舶工程有限公司购买23000T...
